Yolo mask detection in video

In [7]:
import cv2
import numpy as np
# change file location according to your drive
weight_path='C:\\Users\\Vishal\\AIS_vision100.ai\\mask_detection\\Yolov3_mask_detection.weights' #weight file path
cfg_path='C:\\Users\\Vishal\\AIS_vision100.ai\\mask_detection\\yolov3_testing.cfg' # Configuration file path 
classes = ['Mask Found','Mask cover 50 %','Mask Not Found']
threshold=0.6
font = cv2.FONT_HERSHEY_PLAIN
font_=cv2.FONT_HERSHEY_COMPLEX
net = cv2.dnn.readNet(weight_path,cfg_path)

cap = cv2.VideoCapture(0)
while True:
    _, img = cap.read()
    height, width, _ = img.shape

    blob = cv2.dnn.blobFromImage(img, 1/255, (416, 416), (0,0,0), swapRB=True, crop=False)
    net.setInput(blob)
    output_layers_names = net.getUnconnectedOutLayersNames()
    layerOutputs = net.forward(output_layers_names)

    boxes = []
    confidences = []
    class_ids = []

    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > threshold:
                center_x = int(detection[0]*width)
                center_y = int(detection[1]*height)
                h= int(detection[2]*width)
                w= int(detection[3]*height*0.8)

                x = int(center_x - w/2)
                y = int(center_y - h/2)

                boxes.append([x, y, w, h])
                confidences.append((float(confidence)))
                class_ids.append(class_id)
    cv2.putText(img,str('AIS Private Ltd'), (100,50), font_, 2, (250,250,250), 2)
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    if len(indexes)>0:
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = str(round(confidences[i],2))
            #############
            if label==classes[2]:
                cv2.rectangle(img,(x,y), (x+h,y+w), (0,0,255), 1)
                cv2.putText(img,str(label), (x,y-5), font, 1, (0,0,255), 2)
                cv2.putText(img,str(float(confidence)*100)+'%', (x,y-30), font, 1, (0,0,255), 2)
            if label==classes[0]:
                cv2.rectangle(img, (x,y), (x+h,y+w), (255,0,0), 1)
                cv2.putText(img,str(label), (x,y-5), font, 1, (255,0,0), 2)
                cv2.putText(img,str(float(confidence)*100)+'%', (x,y-30), font, 1, (255,0,0), 2)
            if label==classes[1]:
                cv2.rectangle(img, (x,y), (x+h,y+w), (0,255,0), 1)
                cv2.putText(img,str(label), (x,y-5), font, 1, (0,255,0), 2)
                cv2.putText(img,str(float(confidence)*100)+'%', (x,y-30), font, 1, (0,255,0), 2)
                
    cv2.imshow('Image', img)
    key=cv2.waitKey(1)
    if key ==ord('v'): 
        break 
cap.release()
cv2.destroyAllWindows()